# Lab 6: TensorQTL

In eQTL analysis, we test association for each gene in transcriptome against a set of variants.
Typically, for cis-eQTL analysis, one gene could have thousands of variants to test against. 
Matrix eQTL @matrixqtl was developed to create a computationally less burdensome analysis for eQTL identification. 
Compared to other existing tools for QTL analysis, Matrix eQTL is **orders of magnitude faster** in analysis due to specific preprocessing and its use of large matrix operations for the computationally difficult aspects of the system. 

To compare between genes which habor different LD structures and different number of testing variants, the statistical test underlying cis-eQTL requires permutation.
So it is computationally intensive even with matrixQTL. 
To overcome the computational burden of permutation, FastQTL @fastqtl proposed an approximate simulation scheme which is efficient for large-scale transcriptome-wide analysis, *i.e.* Geuvadis & GTEx.

In the past few years, GPU has been widely adapted for many computations. 
TensorQTL @tensorqtl was recently developed which is capable of running on both CPU and GPU. 
And when GPU is enabled, it runs 100 times faster than CPU. 
Today, we will learn to use `tensorqtl` with CPU. (And to run on GPU needs only few more setup steps with the same command!)
By the end of the lab you should be able to:

- **Understand the types of files required for tensorQTL**  
- **Interpret the results of tensorQTL**  


# tensorQTL

## Input files for eQTL analysis

* **Phenotype**: a matrix representing gene expression levels in each individual (gene x individual)
* **Covariate**: a matrix representing value of covariate values in each individual (covariate x individual)
* **Genotype**: a matrix representing genotype dosage (effect allele) for each variant and individual (in plink format)

Note that we need to know the genomic position of the gene (say the position of transcription start site, TSS) since we test cis-eQTL for nearby variants only. The example data for this lab is at `lab6`:

## Input files description

* **Phenotype**: `GEUVADIS.chr22.expression.bed.gz`
* **Covariate**: `GEUVADIS.445_samples.covariates.txt`
* **Genotype**: `GEUVADIS.hg38.chr22.*`

*Problem 1*

How many covariates are in the example data?

## Compute nominal p-value in cis-eQTL analysis

Nominal p-value is the observed p-value under linear model $\tilde{Y} \sim X$, where $\tilde{Y}$ is residual expression level after regressing out covariates and $X$ is the genotype dosage of a variant of interest.

Let's compute nomimal p-value for all cis-eQTL candidates. 
Here we define cis-window as 10kb surrounding TSS (both sides).

In [1]:
cd lab6
mkdir  -output
pytensorqtl=~/mambaforge/envs/cumc_statgen/bin/tensorqtl

$pytensorqtl \
  --covariates data/lab6/GEUVADIS.445_samples.covariates.txt \
  --window 10000 \
  --mode cis_nominal \
  data/lab6/GEUVADIS.hg38.chr22 \
  data/lab6/GEUVADIS.chr22.expression.bed.gz \
  output/cis_nominal

Unable to load extension: pydevd_plugins.extensions.types.pydevd_plugin_pandas_types
Exception in thread Heartbeat:
Traceback (most recent call last):
  File "/home/leibowen/miniconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/leibowen/miniconda3/lib/python3.9/site-packages/ipykernel/heartbeat.py", line 97, in run
    self._bind_socket()
  File "/home/leibowen/miniconda3/lib/python3.9/site-packages/ipykernel/heartbeat.py", line 77, in _bind_socket
    self._try_bind_socket()
  File "/home/leibowen/miniconda3/lib/python3.9/site-packages/ipykernel/heartbeat.py", line 64, in _try_bind_socket
    return self.socket.bind('%s://%s' % (self.transport, self.ip) + c + str(self.port))
  File "/home/leibowen/miniconda3/lib/python3.9/site-packages/zmq/sugar/socket.py", line 229, in bind
    super().bind(addr)
  File "zmq/backend/cython/socket.pyx", line 562, in zmq.backend.cython.socket.Socket.bind
  File "zmq/backend/cython/checkrc.pxd", line 28, in z

mkdir: invalid option -- 'o'
Try 'mkdir --help' for more information.
[Jun 17 12:57:14] Running TensorQTL: cis-QTL mapping
  * WARNING: using CPU!
  * reading phenotypes (data/lab6/GEUVADIS.chr22.expression.bed.gz)
  * cis-window detected as position ± 10,000
  * reading covariates (data/lab6/GEUVADIS.445_samples.covariates.txt)
  * loading genotypes
Mapping files:   0%|                                      | 0/3 [00:00<?, ?it/s]/home/leibowen/mambaforge/envs/cumc_statgen/lib/python3.12/site-packages/tensorqtl/genotypeio.py:147: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.bim, self.fam, self.bed = read_plink(plink_prefix_path, verbose=verbose)
Mapping files:  33%|██████████                    | 1/3 [00:00<00:00,  3.18it/s]/home/leibowen/mambaforge/envs/cumc_statgen/lib/python3.12/site-packages/tensorqtl/genotypeio.py:147: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv 

# 这一步得到的结果和dowload的文件一样


*Problem 2*

From the logging message of tensorqtl run, how many genes are being analyzed? 

The output contains all variant/gene pairs being test regardless of significance. 
So, it will be huge amount of data in practice. 
The output file is in `parquet` format, which is a binary format but it gives better I/O performance as comparing to human-readable text file. 
We've provided a tiny python script to convert `parquet` file to text table in `txt.gz`.

In [ ]:
parquet2table.py \
  --parquet output/cis_nominal.cis_qtl_pairs.chr22.parquet \
  --output  data/lab6/output/cis_nominal.cis_qtl_pairs.chr22.txt.gz

# Q2 parquet2table.py这个脚本哪里来的？应该是自己写的

*Problem 3*

How many variant/gene pairs are being tested and reported?

*Problem 4*

Which genes has the strongest association?


## Perform cis-eQTL analysis with adaptive permutation

If we'd like to identify eGene (gene that is significantly regulated by genetic variation), like we've mentioned above, we need to perform permutation to obtain gene-level p-value. 
Here is how it can be done using `tensorqtl`.

In [ ]:
R -e 'if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager"); BiocManager::install(c("qvalue"), force = TRUE)'
# R -e 'if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager"); BiocManager::install(c("qvalue", "beachmat", "curl", "data.table", "dqrng", "ebnm", "GBJ", "ieugwasr", "lava", "mashr", "MendelianRandomization", "pgenlibr", "RcppParallel", "S4Arrays", "SparseArray", "zlibbioc"), force = TRUE)'


In [1]:
cd lab6
pytensorqtl=~/mambaforge/envs/cumc_statgen/bin/tensorqtl
$pytensorqtl \
  --covariates data/lab6/GEUVADIS.445_samples.covariates.txt \
  --window 10000 \
  --mode cis \
  data/lab6/GEUVADIS.hg38.chr22 \
  data/lab6/GEUVADIS.chr22.expression.bed.gz \
  output/cis

Unable to load extension: pydevd_plugins.extensions.types.pydevd_plugin_pandas_types


[Jun 17 14:20:20] Running TensorQTL: cis-QTL mapping
  * WARNING: using CPU!
  * reading phenotypes (data/lab6/GEUVADIS.chr22.expression.bed.gz)
  * cis-window detected as position ± 10,000
  * reading covariates (data/lab6/GEUVADIS.445_samples.covariates.txt)
  * loading genotypes
Mapping files:   0%|                                      | 0/3 [00:00<?, ?it/s]/home/leibowen/mambaforge/envs/cumc_statgen/lib/python3.12/site-packages/tensorqtl/genotypeio.py:147: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.bim, self.fam, self.bed = read_plink(plink_prefix_path, verbose=verbose)
Mapping files:  33%|██████████                    | 1/3 [00:00<00:00,  2.79it/s]/home/leibowen/mambaforge/envs/cumc_statgen/lib/python3.12/site-packages/tensorqtl/genotypeio.py:147: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'

# Q3 qvalue 这个R包需要额外安装

The output is the gene-level statistics obtained from adaptive permutation where each row is for one gene (in txt.gz format).
To obtain eGene as FDR 10%, we can collect all genes with `qval` smaller than 0.1. 
To obtain cis-eQTL for these eGenes, we can collect all variant/gene pairs with `pval_nominal` (reported in `cis_nominal` run) smaller than `pval_nominal_threshold`.

*Problem 5*

Which gene has the most significant q-value?

Note, it is possible your own cis output file might lack the q-value column (column 18, called "qval") and pval_nominal_threshold information, if this is the case, simply use the pre-computed `pre_run.cis_qtl.txt` in the data folder.

*Problem 6*

Select a gene with q-value < 0.05, visualize its cis-eQTL results by plotting $-\log(p)$
on y-axis and distance to TSS on x-axis. And put a horizontal line indicating the corresponding `pval_nominal_threshold` of the gene.


**References**:

Ongen, Halit, Alfonso Buil, Andrew Anand Brown, Emmanouil T Dermitzakis, and Olivier Delaneau. 2016. “Fast and Efficient Qtl Mapper for Thousands of Molecular Phenotypes.” Bioinformatics 32 (10). Oxford University Press: 1479–85.

Shabalin, Andrey A. 2012. “Matrix eQTL: Ultra Fast eQTL Analysis via Large Matrix Operations.” Bioinformatics 28 (10). Oxford University Press: 1353–8.

Taylor-Weiner, Amaro, François Aguet, Nicholas J Haradhvala, Sager Gosai, Shankara Anand, Jaegil Kim, Kristin Ardlie, Eliezer M Van Allen, and Gad Getz. 2019. “Scaling Computational Genomics to Millions of Individuals with Gpus.” Genome Biology 20 (1). BioMed Central: 1–5.